In [1]:
import pandas as pd
from collections import defaultdict
import gzip
import numpy as np
import random
from tqdm import tqdm
import time

In [2]:
path = "./small_data/ydata-fp-td-clicks-v2_0.20111002.gz"
dic = defaultdict(list)
with gzip.open(path,"rb") as f:
    lines = f.readlines()
    for idx, line in enumerate(lines):
        elements = line.decode('utf-8').strip().split(" |")
        ts, displayed_arm, reward = elements[0].split(" ")
        feature_idx = np.array(elements[1].split(" ")[1:]).astype("int")-1
        features = np.zeros(136)
        features[feature_idx] = 1
        pools = elements[2:]
        
        dic['displayed_arm'].append(displayed_arm)
        dic['reward'].append(reward)
        dic['features'].append(features)
        dic['pool'].append(pools)

In [3]:
train = pd.DataFrame(dic)
train.head(5)

,displayed_arm,reward,features,pool
0,id-560620,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[id-552077, id-555224, id-555528, id-559744, i..."
1,id-565648,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[id-552077, id-555224, id-555528, id-559744, i..."
2,id-563115,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[id-552077, id-555224, id-555528, id-559744, i..."
3,id-552077,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[id-552077, id-555224, id-555528, id-559744, i..."
4,id-564335,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[id-552077, id-555224, id-555528, id-559744, i..."


In [4]:
arm = train['displayed_arm'].unique()
random.shuffle(arm)
arm

array(['id-562374', 'id-564604', 'id-565589', 'id-566838', 'id-560290',
       'id-566573', 'id-566825', 'id-565648', 'id-565561', 'id-563582',
       'id-565980', 'id-565822', 'id-566587', 'id-566602', 'id-552077',
       'id-566013', 'id-563938', 'id-563787', 'id-563115', 'id-564335',
       'id-566631', 'id-559855', 'id-566689', 'id-565515', 'id-566431',
       'id-559833', 'id-560518', 'id-559744', 'id-566541', 'id-566022',
       'id-565747', 'id-564371', 'id-563643', 'id-566478', 'id-562265',
       'id-560620', 'id-565479', 'id-563846', 'id-564418', 'id-560805',
       'id-565533', 'id-565364', 'id-566726', 'id-566092', 'id-555224',
       'id-566439', 'id-562637', 'id-555528', 'id-565930'], dtype=object)

In [5]:
reward = np.zeros(len(arm))
play = np.zeros(len(arm))
mu = np.zeros(len(arm))
time = 0

In [6]:
iter_ = max(train.index)

In [7]:
while sum(play == 1) != len(arm):
    tmp_idx = np.argmin(play)
    play[tmp_idx] += 1
    if train['displayed_arm'].iloc[time] == arm[tmp_idx]:
        reward[tmp_idx] += int(train['reward'].iloc[time])
    else:
        reward[tmp_idx] += 0
    time += 1
    mu = reward/play

while time < iter_:
    action = np.argmax(mu + (2*np.log(time))/play)
    play[action] += 1
    if train['displayed_arm'].iloc[time] == arm[action]:
        reward[action] += int(train['reward'].iloc[time])
    else:
        reward[action] += 0
    time += 1
    mu = reward/play

C:\Users\Byeongwoo\AppData\Local\Temp\ipykernel_10872\420177933.py:9: RuntimeWarning: invalid value encountered in divide
  mu = reward/play


In [8]:
reward_result = reward.astype(int)
arm_reward = dict(zip(arm, reward_result))
arm_reward

{'id-562374': 0,
 'id-564604': 4,
 'id-565589': 30,
 'id-566838': 0,
 'id-560290': 18,
 'id-566573': 0,
 'id-566825': 0,
 'id-565648': 10,
 'id-565561': 16,
 'id-563582': 33,
 'id-565980': 1,
 'id-565822': 330,
 'id-566587': 0,
 'id-566602': 0,
 'id-552077': 1,
 'id-566013': 20,
 'id-563938': 2,
 'id-563787': 0,
 'id-563115': 75,
 'id-564335': 7,
 'id-566631': 0,
 'id-559855': 17,
 'id-566689': 0,
 'id-565515': 95,
 'id-566431': 2378,
 'id-559833': 0,
 'id-560518': 6,
 'id-559744': 2,
 'id-566541': 0,
 'id-566022': 23,
 'id-565747': 137,
 'id-564371': 4,
 'id-563643': 48,
 'id-566478': 0,
 'id-562265': 0,
 'id-560620': 3,
 'id-565479': 23,
 'id-563846': 65,
 'id-564418': 19,
 'id-560805': 11,
 'id-565533': 22,
 'id-565364': 12,
 'id-566726': 0,
 'id-566092': 13,
 'id-555224': 51,
 'id-566439': 0,
 'id-562637': 0,
 'id-555528': 12,
 'id-565930': 18}